In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\keb7\AppData\Local\Temp\ipykernel_62116\1140549349.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1662405452.0, "C:\\Users\\keb7\\AppData\\Local\\Programs\\Python\\Python310\\lib\\stat.py

In [2]:
import pandas as pd
import plotly.graph_objects as go
import statsmodels.formula.api as smf
from pandas_datareader import DataReader as pdr

ret = pd.read_csv("files/rets.csv", index_col="date")
ret = 1.5*ret.best - 0.5*ret.worst

ff = pdr("F-F_Research_Data_Factors", "famafrench", start=2005)[0]/100
ff.index = ff.index.astype(str)
mkt_rf = ff["Mkt-RF"]
rf = ff["RF"]

df = pd.concat((ret, mkt_rf, rf), axis=1)
df.columns = ["ret", "mkt_rf", "rf"]
df["ret_rf"] = df.ret - df.rf
df["mkt"] = df.mkt_rf + df.rf
df.index.name = "date" 
df = df.reset_index()
df = df.dropna()

result = smf.ols("ret_rf~mkt_rf", df).fit()
beta = result.params["mkt_rf"]
mkt = df.rf + beta*df.mkt_rf 
active = df.ret - mkt

trace1 = go.Scatter(
    x = df.date,
    y = (1+df.ret).cumprod(),
    name = "150-50 total",
)
trace2 = go.Scatter(
    x = df.date,
    y = (1+active).cumprod(),
    name = "150-50 active",
)
trace3 = go.Scatter(
    x = df.date,
    y = (1+mkt).cumprod(),
    name = "150-50 market",
)
fig = go.Figure()
for trace in [trace1, trace3, trace2]:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_title="Accumulation",
    xaxis_title_font = {"size":18},
    template="plotly_white",
    yaxis_tickprefix="$",
    yaxis_tickformat=".2f",
    height=600,  
    width=1000,
    legend=dict(
      x = 0.01,
      y = 0.99
    ),
    font_size=16
)

fig.show()